In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 16
### Team Member Names: Matthew Gong, Arya Patel, Saihej Singh
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [21]:
possibleTickers = pd.read_csv("Tickers_Example.csv")
tickers = []
tickers_hist = {}
tickers_long_hist = {}
minVolume = 150000
allowed_currencies = ["USD", "CAD"]
start_date = "2023-01-01"
end_date = "2023-10-31"
long_start_date = "2018-10-31"
threshold_day = 18
days_in_month = []

for ticker in possibleTickers[possibleTickers.columns.values[0]]:
    currTicker = yf.Ticker(ticker)
    currHist = currTicker.history(start=start_date, end=end_date)
    currFastInfo = currTicker.fast_info

    if currHist.empty:
        print(ticker, "is delisted.")
    elif currTicker in tickers:
        print(ticker, "is a duplicate.")
    else:
        tickers_hist[ticker] = currHist
        tickers_long_hist[ticker] = currTicker.history(start=long_start_date, end=end_date)

        group_months = currHist.reset_index().groupby(pd.Grouper(key='Date', freq='M'))
        dfs = [group for _, group in group_months]
        filtered_dfs = []
        for i, df in enumerate(dfs):
            #recombine all the months with more trading days than set threshold
            if len(df.index) >= threshold_day:
                days_in_month.append(len(df.index))
                filtered_dfs.append(df)
        currHist_updated = pd.concat(filtered_dfs)
        currHist_updated.set_index("Date", inplace=True)

        if ticker in tickers:
            print(ticker, "is duplicate")
        elif currFastInfo["currency"] not in allowed_currencies:
            print(ticker, "is not in USD or CAD")
        elif currHist["Volume"].mean() * (np.mean(days_in_month)) < minVolume:
            print(ticker, "only has a volume of", currFastInfo["lastVolume"], "and average days of", np.mean(days_in_month))
        else:
            tickers.append(ticker)

print(possibleTickers.values)

AGN: No timezone found, symbol may be delisted


AGN is delisted.


CELG: No timezone found, symbol may be delisted


CELG is delisted.


MON: No timezone found, symbol may be delisted


MON is delisted.


RTN: No timezone found, symbol may be delisted


RTN is delisted.
[['ABBV']
 ['ABT']
 ['ACN']
 ['AGN']
 ['AIG']
 ['AMZN']
 ['AXP']
 ['BA']
 ['BAC']
 ['BIIB']
 ['BK']
 ['BLK']
 ['BMY']
 ['C']
 ['CAT']
 ['CELG']
 ['CL']
 ['KO']
 ['LLY']
 ['LMT']
 ['MO']
 ['MON']
 ['MRK']
 ['PEP']
 ['PFE']
 ['PG']
 ['PM']
 ['PYPL']
 ['QCOM']
 ['RTN']
 ['RY.TO']
 ['SHOP.TO']
 ['T.TO']
 ['TD.TO']
 ['TXN']
 ['UNH']
 ['UNP']
 ['UPS']
 ['USB']
 ['YCBD']]


In [22]:
marketTicker = yf.Ticker("^GSPC")
marketHist = marketTicker.history(start=long_start_date, end=end_date)

beta = {}

def beta_volatility(ticker):
    prices = pd.DataFrame(tickers_long_hist[ticker]["Close"])
    prices.columns = [ticker]
    prices["Market"] = marketHist['Close']

    monthly_change = prices.resample("M").ffill().pct_change()
    monthly_change = monthly_change.drop(index=monthly_change.index[0])

    return (monthly_change.cov()/monthly_change["Market"].var()).iat[0, 1]


for ticker in tickers:
    beta[ticker] = beta_volatility(ticker)

sorted(beta.items(), key=lambda x:x[1], reverse=True)


C:\Users\matth\AppData\Local\Temp\ipykernel_180\48051138.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffill().pct_change()
C:\Users\matth\AppData\Local\Temp\ipykernel_180\48051138.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffill().pct_change()
C:\Users\matth\AppData\Local\Temp\ipykernel_180\48051138.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffil

[('YCBD', 2.1401685163092337),
 ('SHOP.TO', 1.8105683661552798),
 ('C', 1.5566634647492017),
 ('BA', 1.4762119826728957),
 ('BAC', 1.3851004801566917),
 ('PYPL', 1.379281690066955),
 ('BLK', 1.2835911068998493),
 ('ACN', 1.23045059202993),
 ('QCOM', 1.2167444255405166),
 ('AXP', 1.2146618730775025),
 ('AMZN', 1.1719942897819302),
 ('UPS', 1.104947789428036),
 ('UNP', 1.1025185401495559),
 ('BK', 1.095339131727691),
 ('CAT', 1.0879436753753293),
 ('AIG', 1.0351018552031348),
 ('USB', 1.0267302590287006),
 ('TXN', 1.0168278673298132),
 ('PM', 0.776693239874068),
 ('ABT', 0.6933949493135634),
 ('MO', 0.6535228633258505),
 ('TD.TO', 0.6386754199351966),
 ('UNH', 0.6321140262217283),
 ('PFE', 0.6305321995674552),
 ('LMT', 0.5987252905970897),
 ('KO', 0.5908107221173703),
 ('PEP', 0.5726422638704429),
 ('RY.TO', 0.5664455168681224),
 ('ABBV', 0.5070220958555529),
 ('PG', 0.4747910915156738),
 ('CL', 0.46066659442551405),
 ('T.TO', 0.4596691542353901),
 ('BMY', 0.3853920943321375),
 ('MRK', 0

In [ ]:
def correlation(stock1, stock2):
    stock1Hist = tickers_hist[stock1].copy()
    stock2Hist = tickers_hist[stock2].copy()

    prices = pd.DataFrame(stock1Hist["Close"])
    prices.columns = [stock1]
    prices[stock2] = stock2Hist["Close"]
    
    monthly_returns = prices.resample('M').ffill().pct_change()
    monthly_returns.drop(index=monthly_returns.index[0], inplace=True)

    return monthly_returns.corr()

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.